In [1]:
import pandas as pd
import requests
import urllib.request
import time
from urllib.request import Request
from bs4 import BeautifulSoup
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize

/home/angel/miniconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [1]:
from openpyxl import load_workbook
wb = load_workbook(filename='LinkNYC_Coverage_Tracker.xlsx')
ws = wb['2019'] # ws is now an IterableWorksheet

# iterate thru all cells and if hyperlink found attempt modification of cell
for row in ws.rows:
    for cell in row:
        try:
            if len(cell.hyperlink.target)  > 0:
                cell.value = "".join([cell.value,"|",cell.hyperlink.target])
                # Join cell.value and hyperlink target into string (optionally just assign the hyperlink.target to the cell.value
        except:
            pass

# save workbook to temp .xlsx (I could not manage to read from buffer...) .
wb.save("temp.xlsx")

# read with pandas 
data = pd.read_excel("temp.xlsx")

# take DataSeries and rsplit by "|" and expand to 2 columns
hyper = (data.Headline.str.rsplit("|", expand=True))

#set labels
hyper.columns=["Label","Hyperlink"]

# join them back to dataframe on index
data = data.join(hyper, how="left")

In [2]:
hyperdata = data[['Date','Area of Business','Headline','Sentiment (Positive, Negative, Neutral)','Label', 'Hyperlink']]

In [3]:
hyperdata.dropna(inplace = True)

/home/angel/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [4]:
wb1 = load_workbook(filename='LinkNYC_Coverage_Tracker.xlsx')
ws1 = wb1['2018']

In [5]:
ws_dict = []
sheetName = ['2018','2017','2016','2015','Greatest Hits']
for sheetn in sheetName:
    ws_dict.append(pd.read_excel('LinkNYC_Coverage_Tracker.xlsx', sheetname=sheetn))

/home/angel/miniconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [6]:
hyperlink = []
for i in range(len(ws_dict)):
    hyperlink.append(ws_dict[i].URL)

In [27]:
corpus_2018 = []
for i in range(len(hyperlink[0])):
    try:
        req = Request(hyperlink[0][i], headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(webpage)
        for script in soup(["script", "style"]):
            script.extract() 
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        corpus_2018.append(text)
    except:
        pass

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [36]:
corpus = [x.encode('utf-8') for x in corpus_2018]

In [39]:
with open("text_2018.txt", "w") as output:
    output.write(str(corpus))

In [41]:
corpus_2017 = []
for i in range(len(hyperlink[1])):
    try:
        req = Request(hyperlink[1][i], headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(webpage)
        for script in soup(["script", "style"]):
            script.extract() 
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        corpus_2017.append(text)
    except:
        pass
corpus2017 = [x.encode('utf-8') for x in corpus_2017]
with open("text_2017.txt", "w") as output:
    output.write(str(corpus2017))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [13]:
corpus_2015 = []
for i in range(len(hyperlink[3])):
    try:
        req = Request(hyperlink[3][i], headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(webpage)
        for script in soup(["script", "style"]):
            script.extract() 
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        corpus_2015.append(text)
    except:
        pass
corpus2015 = [x.encode('utf-8') for x in corpus_2015]
with open("text_2015.txt", "w") as output:
    output.write(str(corpus2015))

In [14]:
Greatest_Hits = []
for i in range(len(hyperlink[4])):
    try:
        req = Request(hyperlink[4][i], headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urllib.request.urlopen(req).read()
        soup = BeautifulSoup(webpage)
        for script in soup(["script", "style"]):
            script.extract() 
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        Greatest_Hits.append(text)
    except:
        pass
Greatest_Hits = [x.encode('utf-8') for x in Greatest_Hits]
with open("Greatest_Hits.txt", "w") as output:
    output.write(str(Greatest_Hits))

In [7]:
def prepocess(text):
    return re.sub(r"[^a-zA-Z0-9]+", ' ', re.sub(r'\d+', '', text.lower()).translate(str.maketrans('', '', string.punctuation)).strip())

In [2]:
with open('text_2015.txt', 'r') as f:
    text = f.readlines()

In [50]:
with open('Greatest_Hits.txt', 'r') as f:
    text_grehit = f.readlines()

In [57]:
with open('text_2017.txt', 'r') as f:
    text_2017 = f.readlines()

In [62]:
with open('text_2018.txt', 'r') as f:
    text_2018 = f.readlines()

In [6]:
not_want = ['de','xexx','xcxa','u','le','la','en','xexxbnback','also','said','york']

In [6]:
def top_model(corpus):
    corpus_topic = []
    for sent in corpus:
        corpus_topic.append(prepocess(corpus[0]))
    stop_words = set(stopwords.words('english'))
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(str(corpus_topic))
    result = [i for i in tokens if not i in stop_words]
    result = [i for i in result if not i in not_want]
    lemmatizer=WordNetLemmatizer()
    preprocess_text = []
    for word in result:
        preprocess_text.append(lemmatizer.lemmatize(word))
    dictionary = gensim.corpora.Dictionary([d.split() for d in preprocess_text])
    bow_corpus = [dictionary.doc2bow(doc) for doc in[d.split() for d in preprocess_text]]
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 6, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)
    topics = lda_model.print_topics(num_words=5)
    for topic in topics:
        print(topic)

In [5]:
def viz_model(text):
    corpus_topic = []
    for sent in text:
        corpus_topic.append(prepocess(text[0]))
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(str(corpus_topic))
    result = [i for i in tokens if not i in stop_words]
    result = [i for i in result if not i in not_want]
    lemmatizer=WordNetLemmatizer()
    preprocess_text = []
    for word in result:
        preprocess_text.append(lemmatizer.lemmatize(word))
    dictionary = gensim.corpora.Dictionary([d.split() for d in preprocess_text])
    corpus = [dictionary.doc2bow(doc) for doc in[d.split() for d in preprocess_text]]
    ldamodel = LdaModel(corpus=corpus, num_topics=6, id2word=dictionary)
    pyLDAvis.enable_notebook()
    pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [66]:
pyLDAvis.save_html(pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary), 'lda_2018.html')

/home/angel/miniconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [17]:
top_model(text)

(0, '0.058*"wifi" + 0.049*"new" + 0.022*"public" + 0.020*"day" + 0.020*"booth"')
(1, '0.011*"business" + 0.010*"good" + 0.008*"would" + 0.007*"find" + 0.007*"home"')
(2, '0.015*"get" + 0.011*"news" + 0.010*"company" + 0.009*"digital" + 0.009*"gigabit"')
(3, '0.011*"finally" + 0.010*"million" + 0.009*"hotspot" + 0.009*"say" + 0.009*"right"')
(4, '0.026*"free" + 0.019*"first" + 0.015*"access" + 0.012*"service" + 0.011*"next"')
(5, '0.034*"city" + 0.024*"phone" + 0.023*"hub" + 0.020*"kiosk" + 0.017*"woman"')


In [84]:
top_model(text_grehit)

(0, '0.095*"new" + 0.024*"wifi" + 0.020*"free" + 0.012*"call" + 0.011*"technology"')
(1, '0.011*"street" + 0.011*"use" + 0.010*"network" + 0.009*"user" + 0.008*"one"')
(2, '0.058*"city" + 0.013*"first" + 0.013*"digital" + 0.011*"public" + 0.010*"news"')
(3, '0.030*"linknyc" + 0.018*"phone" + 0.017*"display" + 0.016*"people" + 0.013*"year"')
(4, '0.066*"kiosk" + 0.024*"time" + 0.018*"make" + 0.018*"company" + 0.014*"service"')
(5, '0.016*"information" + 0.015*"main" + 0.014*"link" + 0.010*"business" + 0.010*"photo"')


In [85]:
top_model(text_2017)

(0, '0.025*"pm" + 0.015*"time" + 0.015*"nyc" + 0.014*"june" + 0.013*"news"')
(1, '0.016*"free" + 0.015*"ny" + 0.014*"make" + 0.011*"use" + 0.011*"toronto"')
(2, '0.015*"data" + 0.014*"park" + 0.013*"get" + 0.013*"work" + 0.012*"business"')
(3, '0.081*"city" + 0.079*"new" + 0.018*"linknyc" + 0.015*"street" + 0.010*"day"')
(4, '0.023*"kiosk" + 0.017*"year" + 0.016*"smart" + 0.015*"company" + 0.015*"technology"')
(5, '0.020*"one" + 0.018*"service" + 0.017*"public" + 0.017*"u" + 0.015*"project"')


In [11]:
top_model(text_2018)

(0, '0.021*"data" + 0.018*"free" + 0.018*"linknyc" + 0.014*"medium" + 0.013*"puerto"')
(1, '0.079*"city" + 0.013*"business" + 0.011*"privacy" + 0.011*"would" + 0.011*"need"')
(2, '0.022*"service" + 0.019*"smart" + 0.015*"phone" + 0.010*"way" + 0.010*"june"')
(3, '0.044*"kiosk" + 0.026*"wifi" + 0.026*"public" + 0.022*"people" + 0.017*"one"')
(4, '0.075*"new" + 0.022*"digital" + 0.020*"technology" + 0.019*"company" + 0.018*"u"')
(5, '0.015*"news" + 0.014*"information" + 0.010*"policy" + 0.009*"community" + 0.009*"work"')


In [8]:
viz_model(text)

/home/angel/miniconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
